In [1]:
pip list

Package                   Version                              Editable project location
------------------------- ------------------------------------ -------------------------------------------------
absl-py                   0.15.0
accelerate                0.33.0
aiosignal                 1.3.1
aiosqlite                 0.19.0
annotated-types           0.6.0
anyio                     4.1.0
appdirs                   1.4.4
argon2-cffi               21.1.0
arrow                     1.3.0
astunparse                1.6.3
async-lru                 2.0.4
attrs                     23.1.0
Automat                   20.2.0
Babel                     2.13.1
backcall                  0.2.0
bcrypt                    3.2.0
beautifulsoup4            4.10.0
beniget                   0.4.1
bleach                    4.1.0
blinker                   1.4
bottle                    0.12.19
Bottleneck                1.3.2
Brotli                    1.0.9
cachetools                5.0.0
certifi               

In [2]:
import re

import torch
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

In [3]:
model_name = "deepseek-ai/DeepSeek-Prover-V1.5-RL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLM(model=model_name, max_num_batched_tokens=8192, seed=1, trust_remote_code=True)

/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-30 22:06:24 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=1)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-30 22:06:25 utils.py:608] Found nccl from library /home/ubuntu/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 09-30 22:06:25 selector.py:28] Using FlashAttention backend.
INFO 09-30 22:06:26 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 09-30 22:06:44 model_runner.py:173] Loading model weights took 12.8725 GB
INFO 09-30 22:06:45 gpu_executor.py:119] # GPU blocks: 2858, # CPU blocks: 546
INFO 09-30 22:06:51 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-30 22:06:51 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-30 22:06:55 model_runner.py:1057] Graph capturing fin

In [4]:
prompt = r'''Complete the following Lean 4 code with short explanatory comments:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''

In [5]:
sampling_params = SamplingParams(
    temperature=1.7,
    max_tokens=500,
    top_p=0.95,
    n=10,
    stop = ['\n']
)

In [6]:
def gen(prompt, code_prefix):
    model_inputs = [prompt + code_prefix]

    model_outputs = model.generate(
        model_inputs,
        sampling_params,
        use_tqdm=False,
    )

    output = model_outputs[0]
    outputs = output.outputs
    print(outputs[0].text)
    return outputs

In [7]:
prompt = r'''Complete the following Lean 4 code with short explanatory comments:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  -- Substitute the sequence formula into the given equations
  rw [h₀] at h₁ h₂
'''

options = gen(prompt, code_prefix)

  -- Simplify the equations using the sequence formula


In [8]:
options

[CompletionOutput(index=0, text='  -- Simplify the equations using the sequence formula', token_ids=[207, 1593, 68246, 1904, 254, 8935, 1244, 254, 6905, 9452, 185], cumulative_logprob=-1.9349333941936493, logprobs=None, finish_reason=stop, stop_reason=
 ),
 CompletionOutput(index=6, text='  -- Simplify the equations using the sequence formula', token_ids=[207, 1593, 68246, 1904, 254, 8935, 1244, 254, 6905, 9452, 185], cumulative_logprob=-1.9349333941936493, logprobs=None, finish_reason=stop, stop_reason=
 ),
 CompletionOutput(index=9, text='  -- Simplify the equations using the sequence formula', token_ids=[207, 1593, 68246, 1904, 254, 8935, 1244, 254, 6905, 9452, 185], cumulative_logprob=-1.9349333941936493, logprobs=None, finish_reason=stop, stop_reason=
 ),
 CompletionOutput(index=3, text='  -- Simplify the equations', token_ids=[207, 1593, 68246, 1904, 254, 8935, 185], cumulative_logprob=-2.222748965024948, logprobs=None, finish_reason=stop, stop_reason=
 ),
 CompletionOutput(index

In [9]:
prompt = r'''Complete the following Lean 4 code with explanatory comments:

```lean4
'''

code_prefix = r'''import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  -- Substitute the sequence formula into the given equations
  rw [h₀] at h₁ h₂
  -- Simplify the equations using the given values
  simp_all
  /- Tactic state:
    a r : ℝ
  u : ℕ → ℝ
  h₀ : ∀ (k : ℕ), u k = a * r ^ k
  h₁ : a * r = 2
  h₂ : a * r ^ 3 = 6
  ⊢ a = 2 / √3 ∨ a = -(2 / √3)
  -/
'''

options = gen(prompt, code_prefix)
print("options\n", options)

  -- Solve for a using the simplified equations
options
 [CompletionOutput(index=0, text='  -- Solve for a using the simplified equations', token_ids=[207, 1593, 6121, 313, 327, 245, 1244, 254, 27637, 8935, 185], cumulative_logprob=-1.9694413617253304, logprobs=None, finish_reason=stop, stop_reason=
), CompletionOutput(index=6, text='  -- Solve for a using the simplified equations', token_ids=[207, 1593, 6121, 313, 327, 245, 1244, 254, 27637, 8935, 185], cumulative_logprob=-1.9694413617253304, logprobs=None, finish_reason=stop, stop_reason=
), CompletionOutput(index=9, text='  -- Solve for a using the simplified equations', token_ids=[207, 1593, 6121, 313, 327, 245, 1244, 254, 27637, 8935, 185], cumulative_logprob=-1.9694413617253304, logprobs=None, finish_reason=stop, stop_reason=
), CompletionOutput(index=8, text='  -- Divide the second equation by the first equation to eliminate a', token_ids=[207, 1593, 83804, 254, 1864, 6337, 457, 254, 1022, 6337, 276, 19832, 245, 185], cumulative

In [17]:
comments = [option.text for option in options]
policy = [len(options) - i for i, option in enumerate(options)]
policy = torch.tensor(policy, dtype=torch.float32)
policy /= policy.sum()

In [18]:
comments, policy

(['  -- Solve for a using the simplified equations',
  '  -- Solve for a using the simplified equations',
  '  -- Solve for a using the simplified equations',
  '  -- Divide the second equation by the first equation to eliminate a',
  '  -- Solve for r by dividing the equation for the third term by the equation for the first term',
  '  -- Divide the equation for the fourth term by the equation for the second term',
  '  -- Derive the relationship between r and a using the given terms',
  '  -- Divide the second equation by the first equation to find the relationship between r and a',
  '  -- Introduce a new equation by dividing h₂ by h₁',
  '  -- Use the fact that 2 and 6 are in geometric sequence to find a relationship between a and r'],
 tensor([0.1818, 0.1636, 0.1455, 0.1273, 0.1091, 0.0909, 0.0727, 0.0545, 0.0364,
         0.0182]))